In [1]:
from PyBTLS.py_module.bridge.bridge import Bridge
from PyBTLS.py_module.bridge.flow import LaneFlow, BridgeFlow
from PyBTLS.py_module.parallel.wrapper import BTLSWrapper
from PyBTLS.py_module.traffic.constant_vehicle import ConstantVehicle
from PyBTLS.py_module.traffic.garage import Garage, Kernels
from PyBTLS.py_module.traffic.PyBtlsDefaultTraffic import Auxerre #OLD CODE, BUT NO REPLACEMENT YET
import numpy as np

In [2]:
# Define the bridge
# This is similar to PyCBA syntax
L = [25.0] #m
R = [-1, 0, -1, 0] #Simply supported
bridge = Bridge(L=L, R=R, n_lane=1)

# Define the Load Effects to analyse
# Adding load effects can be done in two ways
# The first way is to use BTLS built-in influence line
# e.g., this is Influence Line 1, which computes moment at midspan
# See BTLS manual for full list of built-in load effects
bridge.add_load_effect(load_effect = 1) 
# Another way to define load effects is by specifiying:
#   load_effect ('M', 'V', or 'R' for moment, shear or reaction)
#   poi         (The point where the load effect is to be measured)
#   step        (The resolution of the IL calculation. Higher resolution is provides more accurare result but will slow down run time)
# Some other available arguments are:
#   lane_factor (The lane_factor. Can be a list for different factors for each lane, or a singular value for same lane_factor for all. default 1.0)
#   POT         (Minimum LE to be included in Peak Over Threshold (POT) analysis. default to 0.0)
bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], step = 0.1)

# Defining the lane flow
# Lane flow can be constructed using:
#   - data frame (NOT recommended as it is prone to error)
#   - csv file
#   - default files
# Construction via data frame is done using the Pandas.DataFrame arguments and syntax. See Pandas documentation for details.
# Construction via csv file require you to supply the following arguments:
#   path        (Path to the csv file, in string)
# Construction via the default files can be done using the load_default() method. The method can take the argument:
#   sample      (The sample number, either 1 or 2. Currently there are two default lane flow definitions available. Default to 1)
lane_1 = LaneFlow()
lane_1.load_default()


# Define the Bridge Flows
# Multiple lanes can be created. This is then added to a BridgeFlow object, which describe the flow of all the lanes in the bridge
# Lane can be added at initialisation as list or BtlsFlow object, or after initialisation via:
#   - add() method or other aliases of the method: add_lane_flow(), append(), extend())
#   - insert(idx, lane_flow) : insert new lane flow into the array
#   - list indexing          : This is similar to regular list indexing, e.g. bridge_flow[1] = lane_2, del bridge_flow[0], etc.
bridge_flow = BridgeFlow(lane_1)

In [3]:
# DEVELOPMENT VERSION
# THIS WILL CHANGE IN THE FUTURE AS THE txt FILES ARE DEPRECATED

# Creating BTLS wrapper
# The bridge needs to then be wrapped in a BTLSWrapper class. This class provides the method of parsing the required input and reading outputs.
# Arguments here are:
# bridge        (A Bridge object)
# num_days      (Number of days for the simulation)
# working_dir   (The working directory where all txt files will be created and read)
btls = BTLSWrapper(bridge = bridge, bridge_flow = bridge_flow, num_days = 5)

# To change analysis types and other options, see the options object
# Set the Simulation mods:
btls.options.program_mode = "Gen & Sim"
# In this case, we're interested in block maxima analysis. We could also do interval statistics to get a quick overview later on
# We can invoke the setiings one by one, e.g.:
#btls.options.analyse_block_max = 1
#btls.options.write_block_max_summary_files = 1
#btls.options.write_block_max_mixed_analysis = 1
# A simpler way to do this is for a set of analysis, you could do:
btls.options.group_analyse_block_maxima = True
btls.options.group_analyse_statistics = True
# Or to turn off an analysis (by default all analysis are off)
btls.options.group_analyse_pot = False

#Other options, e.g. set simulation time step
btls.options.time_step = 0.01
btls.options.write_vehicle_file = 1
btls.options.write_vehicle_file_format = "MON"

# Define the traffic objects
# NEED WORK HERE
#Traffic Vehicle Settings
btls.options.vehicle_generation_model = "Constant"
btls.options.headway_model = "Free Flow"

constant_vehicle = ConstantVehicle()
constant_vehicle.load_default()
btls.constant_vehicle = constant_vehicle

garage = Garage()
garage.load_default()
btls.garage = garage

kernels = Kernels()
kernels.load_default()
btls.kernels = kernels

btls.traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET

# We can then run the analysis. Output will be read automatically
btls.run()

# SAVING AND LOADING
# Currently not working. Its not saving some attributes correctly, e.g. btls.outputs.block_maxima_summary[0].descriptor
# Potentialr ref:
# https://stackoverflow.com/questions/35783898/python-pickle-not-saving-entire-object
# https://stackoverflow.com/questions/18361678/why-cant-pickle-a-list-that-is-an-attribute-of-an-object
#fname = 'Example 1 Bridge Block Maxima Analysis.obj'
#btls.save(fname)
#
#btls = BTLSWrapper
#btls = btls.load(fname)

---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened


1	2	3	4	5	

Flushing buffer of 73652 vehicles at 6/1/0 0:0:10.8933

Simulation complete

Duration of analysis: 3.949 s


# Analysis
The output of the analysis is store in btls.output object For example, if we're interested in the block maxima seperated by number of vehicles in a crossing:

In [4]:
#display function is not needed if you want to display only one output for one cell. Its use here so we don't need multiple cells.
display(btls.outputs.block_maxima_separated_vehicles[3])
display(btls.outputs.block_maxima_separated_vehicles[1])
display(btls.outputs.block_maxima_separated_vehicles[0])
display(btls.outputs.block_maxima_separated_vehicles[2])

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2099.9,26644.3,20.5,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6200
1,1,2,1583.9,4812.1,21.7,1,1 vehicle(s) Vehicle object at 0x7fb1fcab62c0
2,2,1,2099.9,170408.1,20.5,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6350
3,2,2,1583.9,89800.1,21.7,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6410
4,3,1,2099.9,215378.0,20.5,1,1 vehicle(s) Vehicle object at 0x7fb1fcab64d0
5,3,2,1583.9,206404.0,21.7,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6590
6,4,1,2099.9,273371.8,20.5,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6650
7,4,2,1583.9,292949.8,21.7,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6710
8,5,1,2099.9,409400.4,20.5,1,1 vehicle(s) Vehicle object at 0x7fb1fcab67d0
9,5,2,1583.9,352924.0,21.7,1,1 vehicle(s) Vehicle object at 0x7fb1fcab6890


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2572.8,83574.2,21.58,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4520
1,1,2,2015.5,83574.4,26.82,2,2 vehicle(s) Vehicle object at 0x7fb1fcab45e0
2,2,1,2545.4,149463.9,21.60,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4670
3,2,2,1984.5,149464.1,26.98,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4730
4,3,1,2547.1,258480.6,21.54,2,2 vehicle(s) Vehicle object at 0x7fb1fcab47f0
5,3,2,1978.1,258480.8,26.90,2,2 vehicle(s) Vehicle object at 0x7fb1fcab48b0
6,4,1,2535.3,333889.2,21.54,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4970
7,4,2,1966.6,333889.4,26.79,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4a30
8,5,1,2503.6,354205.9,21.66,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4af0
9,5,2,1900.7,354206.1,26.94,2,2 vehicle(s) Vehicle object at 0x7fb1fcab4bb0


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,2365.3,71848.2,31.23,3,3 vehicle(s) Vehicle object at 0x7fb1fca86470
1,1,2,1806.8,71848.2,31.23,3,3 vehicle(s) Vehicle object at 0x7fb1fca85300
2,2,1,2123.3,130812.5,28.59,3,3 vehicle(s) Vehicle object at 0x7fb1fca85270
3,2,2,1551.4,168091.7,28.91,3,3 vehicle(s) Vehicle object at 0x7fb1fca851b0
4,3,1,2497.7,241473.2,28.88,3,3 vehicle(s) Vehicle object at 0x7fb1fca870a0
5,3,2,1753.3,241473.2,28.88,3,3 vehicle(s) Vehicle object at 0x7fb1fca87160
6,4,1,2269.5,263119.0,31.60,3,3 vehicle(s) Vehicle object at 0x7fb1fca87220
7,4,2,1625.4,263119.0,30.43,3,3 vehicle(s) Vehicle object at 0x7fb1fca872e0
8,5,1,1596.1,381183.9,24.24,3,3 vehicle(s) Vehicle object at 0x7fb1fca873a0
9,5,2,1412.6,381183.9,24.24,3,3 vehicle(s) Vehicle object at 0x7fb1fca87460


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,5,1,168.3,347460.9,28.27,4,4 vehicle(s) Vehicle object at 0x7fb1fcab4e50
1,5,2,130.3,347460.9,28.27,4,4 vehicle(s) Vehicle object at 0x7fb1fcab4f10


Suppose we want to see:  
for a 3 vehicles crossing event, the vehicles that creates the most extreme load_effect for Load Effect 1. We want to see their headway at the maximum loading event.  
We can use regular pandas expressions and methods.

In [5]:
# Shorthand for the output object we're interested in
df = btls.outputs.block_maxima_separated_vehicles[0]
# Filter only load_effect 1
df[df['load_effect_num'] == 1]
# Get the critical vehicles
critical_platoon = df["vehicles"][np.argmax(df.max_load_effect)]
#Calculate their headways
arrival_time_in_second = (critical_platoon.second + critical_platoon.minute * 60 + critical_platoon.hour * 3600).to_numpy()
headway = arrival_time_in_second[1:] - arrival_time_in_second[0:-1]

# Display the results
print("Headways of the critical vehicles for 3 vehicles crossing event are:", headway, "seconds")
display(critical_platoon)

Headways of the critical vehicles for 3 vehicles crossing event are: [0.298 0.555] seconds


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,3,1,2010,19,4,32.108,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,3,1,2010,19,4,32.406,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,3,1,2010,19,4,32.961,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Parallelisation Module
For faster calculations, BTLS can utilise multiple cores to do the simulations in parallel. This is done using BTLSParallelWrappper Object.  
We will simulate 1 year of data for 6 bridges of varying spans.

In [6]:
from PyBTLS.py_module.parallel.parallel import BTLSParallelWrapper

In [7]:
span_array = np.array([10.0, 15.0, 20.0, 25.0, 30.0, 35.0])
btls_parallel = BTLSParallelWrapper(num_subprocess = 6)
for i, span in enumerate(span_array):
    L = [span]
    R = [-1, 0, -1, 0] #Simply supported
    bridge = Bridge(L=L, R=R, n_lane=1)

    bridge.add_load_effect(load_effect = 1) 
    bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], step = 0.1)

    lane_1 = LaneFlow()
    lane_1.load_default()

    bridge_flow = BridgeFlow(lane_1)

    btls = BTLSWrapper(bridge = bridge, bridge_flow = bridge_flow, num_days = 5)

    btls.options.program_mode = "Gen & Sim"
    btls.options.group_analyse_block_maxima = True
    btls.options.group_analyse_statistics = True
    btls.options.group_analyse_pot = False
    btls.options.time_step = 0.01
    btls.options.write_vehicle_file = 1
    btls.options.write_vehicle_file_format = "MON"
    btls.options.vehicle_generation_model = "Constant"
    btls.options.headway_model = "Free Flow"
    constant_vehicle = ConstantVehicle()
    constant_vehicle.load_default()
    btls.constant_vehicle = constant_vehicle
    garage = Garage()
    garage.load_default()
    btls.garage = garage
    kernels = Kernels()
    kernels.load_default()
    btls.kernels = kernels
    btls.traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET


    # Instead of running, we add it to the BTLSParallelWrapper object
    btls_parallel.add(btls)


In [11]:
#Run it!
btls_parallel.run();

Starting Kernel 0
Starting Kernel 1
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...
Starting Kernel 2
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


/Storage/PhD/python_packages/btls/btls/py_src/PyBTLS/py_module/parallel/wrapper.py:55: UserWarning: Working directory already existed, and so was deleted
  warnings.warn("Working directory already existed, and so was deleted")
*** Warning: Influence Surface file could not be opened
*** Warning: Influence Surface file could not be opened


Starting Kernel 3
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...
1	Starting Kernel 4
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...


*** Warning: Influence Surface file could not be opened
*** Warning: Influence Surface file could not be opened


Starting Kernel 5
---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...
1	2	

*** Warning: Influence Surface file could not be opened
*** Warning: Influence Surface file could not be opened


---------------------------------------------
Bridge Traffic Load Simulation - C.C. Caprani
                Version 1.3.5			      
---------------------------------------------

Program Mode: 1

Reading influence surface file: ./Input/IS.txt
Reading influence line file: ./Input/IL.txt
Starting simulation...
Day complete...
1	3	2	4	1	5	
3	2	
Flushing buffer of 73849 vehicles at 6/1/0 0:0:0.391528
1	
Simulation complete

Duration of analysis: 1.478 s
4	2	1	3	5	

Flushing buffer of 74201 vehicles at 6/1/0 0:0:0.874797
2	
Simulation complete

Duration of analysis: 2.329 s
4	3	2	5	

Flushing buffer of 74211 vehicles at 6/1/0 0:0:2.8465

Simulation complete

Duration of analysis: 3.017 s
4	3	5	

Flushing buffer of 73950 vehicles at 6/1/0 0:0:4.0392
3	
Simulation complete

Duration of analysis: 3.876 s
4	5	

Flushing buffer of 74371 vehicles at 6/1/0 0:0:3.69459

Simulation complete

Duration of analysis: 4.854 s
4	5	

Flushing buffer of 74374 vehicles at 6/1/0 0:0:6.91576

Simulation complet

In [18]:
#See the outputs!
display(btls_parallel[0].outputs.block_maxima_summary)
display(btls_parallel[0].outputs.block_maxima_summary[0])

[Load effect 0:  5 row(s) of BlockMaximaSummary object at 0x7fb1fca47c70,
 Load effect 1:  5 row(s) of BlockMaximaSummary object at 0x7fb1fca47970]

,index,event_1_truck,event_2_truck
0,1,547.2,546.2
1,2,547.2,544.2
2,3,547.2,547.0
3,4,547.2,548.5
4,5,547.2,546.2


In [19]:
#Example analysis: Print the block max summary of load effect 1 for all the bridges
load_effect_num = 1
for b in btls_parallel:
    print("=============")
    print("Span =", b.bridge.L, "m")
    display(b.outputs.block_maxima_summary[load_effect_num])

Span = [10.0] m


,index,event_1_truck,event_2_truck
0,1,410.4,423.0
1,2,410.4,420.2
2,3,410.4,429.0
3,4,410.4,430.8
4,5,410.4,429.9


Span = [15.0] m


,index,event_1_truck,event_2_truck,event_3_truck
0,1,740.1,824.4,85.4
1,2,740.1,789.0,79.4
2,3,740.1,804.1,86.7
3,4,740.1,843.3,46.3
4,5,740.1,825.9,48.6


Span = [20.0] m


...

,index,event_1_truck,event_2_truck,event_3_truck
0,1,1583.9,1941.9,1648.3
1,2,1583.9,1922.8,1500.1
2,3,1583.9,1963.8,1504.7
3,4,1583.9,2015.0,1453.6
4,5,1583.9,2016.0,1348.1


Span = [30.0] m


,index,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,2008.5,2783.4,2118.9,881.4
1,2,2008.5,2751.5,1920.1,667.1
2,3,2008.5,2782.0,2401.4,1044.5
3,4,2008.5,2717.6,2415.7,0.0
4,5,2008.5,2750.1,2052.4,0.0


Span = [35.0] m


,index,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,2427.6,3477.5,2670.4,222.9
1,2,2427.6,3596.0,2688.4,2479.7
2,3,2427.6,3576.0,3312.5,1082.5
3,4,2427.6,3506.5,3590.9,391.5
4,5,2427.6,3529.0,2500.3,175.7


In [24]:
# DEVELOPMENT NOTES:
# Current available outputs are:

print("=============")
print("Interval statistics (List array):")
display(btls.outputs.interval_statistics)
print("=============")
print("Cumulative statistics:")
display(btls.outputs.cumulative_statistics)
print("=============")
print("Block maxima summary (List array):")
display(btls.outputs.block_maxima_summary)
print("=============")
print("Block maxima separated vehicles (List array):")
display(btls.outputs.block_maxima_separated_vehicles)
print("=============")
print("Block maxima mixed vehicles:")
display(btls.outputs.block_maxima_mixed_vehicles)
print("=============")
print("Generated vehicle file:")
display(btls.outputs.vehicles) #

Interval statistics (List array):


[Load effect 0:  4 row(s) of IntervalStatistics object at 0x7fb1fc77a1a0,
 Load effect 1:  4 row(s) of IntervalStatistics object at 0x7fb1fc77b5e0]

Cumulative statistics:


,load_effect,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck,event_4_truck
0,1,89596,106721,106721,0.16,4692.12,772.89,1212.57,1470319.52,1.47,0.23,73426.0,15228.0,929.0,13.0
1,2,89596,106721,106721,0.24,3596.04,582.51,905.58,820066.50,1.49,0.30,73426.0,15228.0,929.0,13.0


Block maxima summary (List array):


...

[10 x BlockMaximaEvent object at 0x7fb1fc7ed900,
 10 x BlockMaximaEvent object at 0x7fb1fc7ed630,
 10 x BlockMaximaEvent object at 0x7fb1fc7ed7b0,
 10 x BlockMaximaEvent object at 0x7fb1fc7ee830]

Block maxima mixed vehicles:


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,4439.5,85192.2,32.59,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70850
1,1,2,3477.5,85192.2,34.30,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70910
2,2,1,4692.1,125200.3,27.84,2,2 vehicle(s) Vehicle object at 0x7fb1f7b709a0
3,2,2,3596.0,156693.7,34.23,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70a60
4,3,1,4635.8,201999.4,31.74,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70b20
5,3,2,3576.0,201999.5,34.18,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70be0
6,4,1,4655.6,342597.1,31.79,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70ca0
7,4,2,3590.9,342597.3,34.32,3,3 vehicle(s) Vehicle object at 0x7fb1f7b70d60
8,5,1,4555.7,383309.5,27.90,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70e20
9,5,2,3529.0,383309.8,34.25,2,2 vehicle(s) Vehicle object at 0x7fb1f7b70ee0


Generated vehicle file:


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,1,1,2010,0,0,8.295,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,1,1,2010,0,0,12.818,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,1,1,2010,0,0,14.559,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,1,1,2010,0,0,16.306,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,1,1,2010,0,0,17.862,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74369,1001,5,1,2010,23,59,50.643,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74370,1001,5,1,2010,23,59,52.780,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74371,1001,5,1,2010,23,59,55.643,2,0,2039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74372,1001,5,1,2010,23,59,58.880,5,0,45872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
